In [1]:
number_of_shots = 7

In [2]:
experiment_name = "2023.12.06-06_Llama2_13B_API_7-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [3]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

In [4]:
!pip install -q openai --progress-bar off

In [5]:
!pip install -q langchain --progress-bar off

In [6]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [7]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [8]:
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

In [9]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [10]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [11]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN


In [12]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [13]:
data_sel = data #.iloc[54:] #.sample(2)

In [14]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
# cnt = 54

results = []

for index, row in data_sel.iterrows():

  file = open(log_filename, 'a+', encoding='utf-8')
    
  #print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)
  file.write(few_shots)
  file.write('\n')

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'].replace("\n"," - "))
      file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+str(result['choices'][0]['message']['content'].replace("\n"," - "))+"\n")
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
      file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR:"+str(e)+str(result)+"\n")
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_13B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1
  file.close()

  # if cnt==5:
  #   break

0 6668 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 11--- -  - The text does not mention any specific SDGs, but it does mention "subways" and "public transport," which are related to SDG 11 (Sustainable Cities and Communities).
1 6099 0   Sure, I'd be happy to help! Here's the text you provided: -  - Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life -  - Based on the content of the text, I would classify it as: -  - ---SDG 11--- -  - The text does not mention any specific sustainable development goals, but it does describe a beautiful and luxurious vacation in the Maldives, which is a popular tourist destination known for its beaches and ocean views. Therefore, I have classified it under SDG 11, which is focused on sustainable cities and communities.
2 6471 0   Sure! Here's the text you provid

12 1524 0   Sure! Here's the text you provided, with the appropriate SDG label: -  - ---SDG 5--- -  - The text does not mention any of the other SDGs.
13 4660 0   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label would be: -  - ---SDG 3--- -  - The text mentions using smartphones as thermometers to detect fevers, which is related to the goal of ensuring healthy lives and promoting well-being for all at all ages (SDG 3).
14 5077 0   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 0--- -  - The text does not seem to relate to any specific Sustainable Development Goal, as it only mentions a news opening without providing any specific information or context related to the SDGs.
15 995 0   Sure! Here's the text you provided, with the label for each paragraph: -  - ---SDG 11--- -  - The Crescent Hotel, Fort Worth is located at the crossroads of downtown, the world-renowned Cultural District and s

32 3604 0   ---SDG 16---
33 3940 0   ---SDG 0---
34 6228 0   Sure! Here's the text you provided, with my label for the Sustainable Development Goals (SDGs) adoption by all United Nations Member States in 2015: -  - ---SDG 12--- -  - The text discusses the importance of high school athletes being role models for young children and building relationships with them. It highlights the impact of sports on children's self-confidence and encourages athletes to take advantage of sports camps to make a positive difference in their communities. The text does not mention any specific SDGs, but it aligns with SDG 12, which focuses on responsible consumption and production patterns.
35 3148 0   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses the trailer launch of a movie called "Rangabali" and the entertainment and action it promises to deliver. It also mentions the presence of celebrities at the trailer launch ev

55 2055 0   Sure, I'd be happy to help! Here's the text you provided: -  - "Cochlea cell atlas built from single-cell sequencing discovers new cell types, uncovers hidden molecular features Phys.org" -  - Based on the content of the text, I would classify it as: -  - ---SDG 3--- -  - The reason for this classification is that the text mentions "single-cell sequencing" and "molecular features," which are related to the theme of health and well-being (SDG 3). Additionally, the discovery of new cell types and the uncovering of hidden molecular features could potentially lead to new treatments and therapies for various diseases, which aligns with the goal of ensuring healthy lives and promoting well-being for all (SDG 3).
56 2463 0   Sure! I'll do my best to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015. -  - The text to be processed is: -  - "The Falcons have made a pair of promotions in their front office. Acco

73 2631 1   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 1---
74 1250 1   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label would be: -  - ---SDG 8--- -  - This label represents the goal of "Decent Work and Economic Growth," as the text discusses employment rates, wage differentials, and the impact of reforms on employment levels.
75 2799 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 1--- -  - The text discusses the first target of the first SDG proposed by the Open Working Group (OWG) of Member States, which is to "eradicate extreme poverty for all people everywhere" by 2030. This target is related to SDG 1, which aims to end poverty in all its forms.
76 2056 1   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 1---
77 532 1   Sure, I'd be happy to help! Here's the text you provided,

...

913 5970 15   Sure, I'd be happy to help! Here's the input text you provided: -  - "Table 9 below presents a summary of identified challenges and potential response options and links these to case studies presented in this sourcebook. Please note that this table of challenges and response options is not exhaustive and stakeholders may find other more relevant issues within their national contexts. Some respondents used the opportunity to point out methods to further cooperation, including improved coordination of programs of work or projects undertaken by NFPs, and the potential role of national inter-ministerial committees to oversee implementation of biodiversity relevant activities. This can also address capacity needs." -  - Based on the content of the text, I would classify it as: -  - ---SDG 15---
914 4268 15   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 15--- -  - The text discusses the need to protect and maintain the 

936 3947 15   Sure, I'd be happy to help! Based on the text you provided, the best label would be: -  - ---SDG 15--- -  - This label represents the text's focus on the impact of agricultural expansion on forests, which is directly related to SDG 15: Life On Land.
937 3491 15   Sure, I'd be happy to help! Based on the text you provided, the best label would be: -  - ---SDG 15--- -  - This label represents the text's focus on forest management and the financial costs associated with deforestation, which aligns with SDG 15: Life On Land.
938 2926 15   Sure, I'd be happy to help! Based on the text you provided, the best label would be: -  - ---SDG 15---
939 6288 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text describes the expansion of agricultural land in Mongolia, which is leading to a decline in agricultura

957 3934 15   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label would be: -  - ---SDG 15--- -  - This is because the text discusses sustainable forestry and the responsible use of hunting resources, which aligns with the goals of SDG 15: Life On Land.
958 524 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses biodiversity conservation and the status of ecosystems in different regions of Brazil, and mentions the country's efforts to reduce deforestation. These topics are all relevant to SDG 15, which is focused on life on land.
959 4504 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 15---
960 6558 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
961 4896 16   Sure, I'd be happy to

990 5026 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text advocates for expanding general personal jurisdiction over transnational corporations to provide a remedy for extraterritorial business-related human rights violations, which aligns with SDG 16 (peace, justice and strong institutions).
991 2834 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
992 2469 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
993 1609 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16---
994 5916 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16---
995 567 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses the impact of government interventions on the internet, including surveillanc

1014 2723 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
1015 4695 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
1016 6628 16   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label would be: -  - ---SDG 16--- -  - This is because the text discusses the creation of global institutions, international treaties, and international norms, which are all related to the theme of peace, justice, and strong institutions, which is the focus of SDG 16.
1017 4563 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16---
1018 6710 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 10---
1019 1947 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
CPU times: user 7.64 s, sys: 672 ms, total: 8.31 s
Wall time: 42min 27s


In [15]:
files = glob.glob(output_datapath + "*.pkl")
len(files)

1020

In [16]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(725, 5328, 12, "  Sure, I'd be happy to help! Here's the label for the given text:\n\n---SDG 12---")
(396, 3056, 6, "  Sure, I'd be happy to help! Here's the label for the given text:\n\n---SDG 6---")
(988, 2472, 16, "  Sure, I'd be happy to help! Here's the label for the text you provided:\n\n---SDG 12---")
(414, 2416, 6, '  Sure, I\'d be happy to help! Based on the text you provided, the best label would be:\n\n---SDG 4---\n\nThe text mentions "education and participation" (SDG 4) and "safe drinking water, sanitation and hygiene" (SDG 6.a and 6.b), both of which are related to the Sustainable Development Goals adopted by the United Nations in 2015.')
(802, 6675, 13, "  Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:\n\n---SDG 17---\n\nThe text discusses the importance of mobilizing external resources as an upstream measure of effectiveness

In [17]:
len(tuple_list)

1014

In [18]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [19]:
results_processed

,cnt,id,true_label,predicted_label
0,725,5328,12,"Sure, I'd be happy to help! Here's the label..."
1,396,3056,6,"Sure, I'd be happy to help! Here's the label..."
2,988,2472,16,"Sure, I'd be happy to help! Here's the label..."
3,414,2416,6,"Sure, I'd be happy to help! Based on the tex..."
4,802,6675,13,"Sure, I'd be happy to help! Here's the text ..."
...,...,...,...,...
1009,357,747,5,"Sure, I'd be happy to help! Based on the tex..."
1010,540,1607,9,"Sure, I'd be happy to help! Based on the tex..."
1011,332,2861,5,"Sure, I'd be happy to help! Based on the tex..."
1012,888,1089,14,"Sure, I'd be happy to help! Based on the tex..."


In [20]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the text ..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,"Sure! Here's the text you provided, with the..."
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure! Here's the text you provided, with the..."
...,...,...,...,...,...,...,...,...,...,...,...
1009,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label..."
1010,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex..."
1011,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label..."
1012,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label..."


In [21]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the text ...",11
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,"Sure! Here's the text you provided, with the...",10
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label...",4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure! Here's the text you provided, with the...",12
...,...,...,...,...,...,...,...,...,...,...,...,...
1009,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label...",16
1010,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex...",16
1011,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label...",16
1012,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label...",10


In [22]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,7,2,7,7,2,2,2,7,7,7,7
1,69,69,69,69,69,69,69,69,69,69,69
10,50,47,50,50,47,47,47,50,50,50,50
11,79,74,79,79,74,74,74,79,79,79,79
12,81,73,81,81,73,73,73,81,81,81,81
13,80,72,80,80,72,72,72,80,80,80,80
14,58,55,58,58,55,55,55,58,58,58,58
15,60,58,60,60,58,58,58,60,60,60,60
16,66,56,66,66,56,56,56,66,66,66,66


In [23]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [24]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1014
1014
              precision    recall  f1-score   support

           0       0.71      0.08      0.15        59
           1       0.65      0.75      0.70        60
           2       0.93      0.67      0.78        60
           3       0.76      0.92      0.83        60
           4       0.69      0.97      0.81        60
           5       0.90      0.73      0.81        60
           6       0.98      0.72      0.83        57
           7       0.87      0.76      0.81        59
           8       0.58      0.68      0.63        60
           9       0.83      0.63      0.72        60
          10       0.52      0.43      0.47        60
          11       0.70      0.92      0.79        60
          12       0.63      0.85      0.72        60
          13       0.65      0.87      0.74        60
          14       0.90      0.87      0.88        60
          15       0.88      0.88      0.88        60
          16       0.70      0.78      0.74        59
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
